### 必要モジュールの読み込み


In [24]:
import itertools
import numpy as np
import pandas as pd

### パラメータ設定


In [25]:
shimring = [0.03, 0.05]  # 使用するシムリング厚さ [mm]
min_mismatch = 0.000  # 高さ寸法相互差の最小値 [mm]
max_mismatch = 0.190  # 高さ寸法相互差の最大値 [mm]
res_mismatch = 0.001  # 高さ寸法相互差の測定分解能 [mm]
tolerance = 0.015  # 高さ寸法相互差の許容値 [mm]
max_num = 10  # シムリングの最大使用枚数 [-]

### 最適なシムリングの枚数を求める


In [26]:
# 高さ寸法相互差のリストを作成する
def get_optimal_shimring_combination(shimring, mismatch, tolerance, max_num):
    # シムリングの取りうる組み合わせを作成する
    possible_combinations = []
    for c in itertools.product(range(max_num + 1), repeat=len(shimring)):
        if sum(c) <= max_num:
            possible_combinations.append(c)

    # 有効なシムリングの組み合わせを抽出する
    valid_combinations = []
    for combination in possible_combinations:
        # シムリングの合計厚さ
        total_thickness = np.dot(combination, shimring)

        # シムリング挿入後の高さ寸法相互差
        modified_mismatch = mismatch - total_thickness

        # 高さ寸法相互差が許容範囲内のものを抽出
        if -tolerance <= modified_mismatch <= tolerance:
            valid_combinations.append(combination)

    # 最適なシムリングの組み合わせを抽出する
    optimal_combination = min(valid_combinations)

    return optimal_combination

In [27]:
# 結果を保存するためのデータフレームを作成する
columns = (
    ["mismatch"]
    + [f"t{shimring[i]}" for i in range(len(shimring))]
    + ["num_shimring", "modified_mismatch"]
)
df_result = pd.DataFrame([], columns=columns)

df_result

,mismatch,t0.03,t0.05,num_shimring,modified_mismatch


In [28]:
# 結果をデータフレームに保存する
mismatch_list = np.arange(min_mismatch, max_mismatch + res_mismatch, res_mismatch)
for line_number, mismatch in enumerate(mismatch_list):
    # 最適なシムリングの組み合わせを求める
    optimal_combination = get_optimal_shimring_combination(
        shimring, mismatch, tolerance, max_num
    )

    # 結果を保存する
    total_thickness = np.dot(optimal_combination, shimring)
    df_result.loc[line_number] = [
        mismatch,
        *optimal_combination,
        sum(optimal_combination),
        mismatch - total_thickness,  # modified_mismatch
    ]

df_result

,mismatch,t0.03,t0.05,num_shimring,modified_mismatch
0,0.000,0.0,0.0,0.0,0.000
1,0.001,0.0,0.0,0.0,0.001
2,0.002,0.0,0.0,0.0,0.002
3,0.003,0.0,0.0,0.0,0.003
4,0.004,0.0,0.0,0.0,0.004
...,...,...,...,...,...
186,0.186,0.0,4.0,4.0,-0.014
187,0.187,0.0,4.0,4.0,-0.013
188,0.188,0.0,4.0,4.0,-0.012
189,0.189,0.0,4.0,4.0,-0.011


In [29]:
# 結果をEXCELに保存
df_result.to_excel("結果.xlsx", index=False)